In [39]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
import pandas_datareader.data as web


In [2]:
from dotenv import load_dotenv
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2') 

In [3]:
assets = api.list_assets()
tradeable = [asset for asset in assets if asset.tradable]


In [14]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])
asset_info_df.head()

,symbol
0,OIL
1,BKEM
2,BKSB
3,DKNG
4,BKHY


In [45]:
end_date = datetime.now()
start_date = end_date + timedelta(-365)
sp500 = web.DataReader('sp500', 'yahoo' , start_date, end_date)
#df = pd.DataFrame(ticker)
#df.drop(columns = ['High','Low', 'Open', 'Volume', 'Adj Close'], inplace = True)
#df.index = df.index.date
df.head()

RemoteDataError: No data fetched for symbol sp500 using YahooDailyReader

In [7]:
#covarience = returns['TSLA'].cov(returns['SPY'])

In [149]:
main_df = pd.DataFrame()
tickers =  ['TSLA']
timeframe = '1D'
end_date = datetime.now()
start_date = end_date + timedelta(-365)

for count, ticker in enumerate(tickers):
    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df
    df = df[ticker].drop(columns = ['open', 'high', 'low','volume'])
    df.index = df.index.date
    if main_df.empty:
        main_df = df

    else:
        main_df = main_df.join(df)
        #>>> df.set_index('key').join(other.set_index('key'))

main_df.head()

,close
2019-12-03,336.08
2019-12-04,332.99
2019-12-05,330.34
2019-12-06,335.85
2019-12-09,339.40


In [150]:
sp500_df=pd.read_csv('S&P500_04_26_20.csv',parse_dates=True, infer_datetime_format=True)
sp500_df.drop(columns = ['Open','High','Low','Adj Close','Volume'], inplace = True)
sp500_df.rename(columns = {'Close':'sp500_close','Date':'index'}, inplace =True)
sp500_df.set_index('index',inplace=True)
sp500_df.head()

,sp500_close
index,
2019-04-29,2943.030029
2019-04-30,2945.830078
2019-05-01,2923.729980
2019-05-02,2917.520020
2019-05-03,2945.639893


In [151]:
combined_df = pd.concat([main_df,sp500_df], axis='columns', join = 'inner')
combined_df.head()

,close,sp500_close


In [152]:
combined_df.pct_change()

,close,sp500_close
